In [1]:
import sys
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import itertools

sys.path.append('../')

from sys_utils import load_model
from data_model_region import Region
from data_model import Individual
import typing as t
import random
from tqdm import tqdm

In [2]:
checkpoint_path = '../checkpoints_dev'

regions = load_model(
        Region, name=checkpoint_path+"/regions.jsonl"
    )

individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

In [13]:
individuals_filtered = [x for x in individuals if x.regions != None]
individuals_filtered = [x for x in individuals_filtered if x.id.raw_birthcities != None]


In [76]:
df_individuals = [
    {
        "wikidata_id": x.id.wikidata_id,
        "identifier":[y.location for y in x.id.raw_birthcities],
        "region_code": x.regions,
         "range_impact": x.impact_years,
        'cultural_score': x.cultural_score
    }
    for x in individuals_filtered
]

df_individuals = pd.DataFrame(df_individuals)

In [ ]:
df_region = df_individuals.explode('region_code')


In [77]:
re_region = 're_japan'
min_date= 700
max_date = 1400

In [99]:

df_map = df_region[df_region['region_code']==re_region]
df_map = df_map.explode('identifier')
df_map = df_map.drop_duplicates('wikidata_id', keep = 'first').reset_index(drop=True)
df_map = df_map[df_map['identifier']!='nan'].reset_index(drop=True)


df_map['city_longitude'] = df_map['identifier'].apply(lambda x: x.split(' ')[0].split('(')[1]).astype(float)
df_map['city_latitude'] = df_map['identifier'].apply(lambda x: x.split(' ')[1].split(')')[0]).astype(float)

df_map["year"] = df_map["range_impact"].apply(
        lambda x: [year for year in range(int(x[0]), int(x[1]) + 10, 10)]
    )
df_map = df_map.explode("year")

In [110]:
df_map = df_map[(df_map['year']>=min_date)&(df_map['year']<=max_date)]


In [111]:
df_map.to_csv('maps/df_map.csv')

In [100]:

data_loc = df_map.groupby(['city_latitude', 'city_longitude'])['cultural_score'].sum().reset_index()

latitudes = data_loc['city_latitude'].values
longitudes = data_loc['city_longitude'].values
count = data_loc['cultural_score'].values

In [101]:
min_opacity = 0.7
blur = 12
radius = 8
zoom = 4

In [112]:
from ipyleaflet import Map, Heatmap


In [102]:
m = Map(center= data_loc[['city_latitude', 'city_longitude']].mean().to_list(), zoom=zoom)
#m = Map(zoom=3)

heatmap = Heatmap(locations=[[latitudes[i], longitudes[i], count[i]] for i in range(len(data_loc))],
                  radius=radius, min_opacity=min_opacity, blur=blur)

m.add_layer(heatmap)